# Otimizando Modelo de QA - Quantização: estática ou dinâmica.

In [ ]:
modelo_id_qa = 'optimum/roberta-base-squad2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(modelo_id_qa)

In [ ]:
from optimum.onnxruntime import ORTModelForQuestionAnswering

modelo_qa = ORTModelForQuestionAnswering.from_pretrained(modelo_id_qa)

In [ ]:
dir_qa = 'tmp/onnx-quest/'

tokenizer.save_pretrained(dir_qa)
modelo_qa.save_pretrained(dir_qa)

## Inferindo o Modelo

In [ ]:
onnx_qa = pipeline('question-answering', model=modelo_qa, tokenizer=tokenizer, accelerator = 'ort')

In [ ]:
contexto = 'The automotive industry is booming due to the increasing demand for electric vehicles.'
pergunta = 'Why is the automotive industry booming?'

In [ ]:
def pergunta_resposta(modelo_pipe, contexto=contexto, pergunta=pergunta):
  saida = modelo_pipe(context=contexto, question=pergunta)
  return saida

In [ ]:
pergunta_resposta(onnx_qa)

# Otimização: Quantização

In [ ]:
import onnx
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

In [ ]:
quantizador = ORTQuantizer.from_pretrained(modelo_qa)
config_quantizacao = AutoQuantizationConfig.avx512_vnni(is_static=False)

In [ ]:
dir_quant = '/content/tmp/onnx-quant'

In [ ]:
quantizador.quantize(save_dir = dir_quant,
                     quantization_config = config_quantizacao)

## Inferindo Modelo Quantizado

In [ ]:
modelo_quant = ORTModelForQuestionAnswering.from_pretrained(dir_quant,file_name= 'model_quantized.onnx')
tokenizer = AutoTokenizer.from_pretrained(dir_quant)

In [ ]:
onnx_quant = pipeline('question-answering', model=modelo_quant, tokenizer=tokenizer, accelerator='ort')

pergunta_resposta(onnx_quant)

## Avaliando Modelo

### Tamanho

In [ ]:
import os

def obter_tamanho_diretorio(diretorio):
    tamanho_total = 0
    for caminho_diretorio, subdiretorios, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(caminho_diretorio, arquivo)
            if os.path.isfile(caminho_arquivo):
                tamanho_total += os.path.getsize(caminho_arquivo)
    return tamanho_total / (1024 * 1024)

In [ ]:
obter_tamanho_diretorio(dir_qa)

In [ ]:
obter_tamanho_diretorio(dir_quant)

### Performace

In [ ]:
dados_id_qa = 'squad_v2'

In [ ]:
data = load_dataset(dados_id_qa, split = 'validation[:600]')

In [ ]:
def avaliacao(data, model_qa):
  eval = evaluator('question-answering')
  resultados = eval.compute(
      model_or_pipeline=model_qa,
      data=data,
      metric='squad_v2',
      squad_v2_format=True,
  )
  return resultados

In [ ]:
modelo_original = avaliacao(data, onnx_qa)

In [ ]:
modelo_quantizado = avaliacao(data,onnx_quant)

In [ ]:
modelo_original

In [ ]:
modelo_quantizado

# Publicar Modelo

In [ ]:
nome_rep_quant = 'ONNX-quantizado-roberta-base-squad2'

In [ ]:
onnx_quant.push_to_hub(nome_rep_quant)

## Usar Modelo Quantizado

In [ ]:
modelo_quantizado_rep = pipeline('question-answering', 'mirla/ONNX-quantizado-roberta-base-squad2',
                                 accelerator='ort')
pergunta_resposta(modelo_quantizado_rep)